In [22]:
import torch
from transformer_lens import HookedTransformer
from transformer_lens import utils

from functools import partial

from sae_lens import SparseAutoencoder
from sae_lens.toolkit.pretrained_saes import get_gpt2_res_jb_saes

import plotly.express as px

torch.set_grad_enabled(False)


In [40]:
model: HookedTransformer = HookedTransformer.from_pretrained('gpt2-small', device='cpu')
# model: HookedTransformer = HookedTransformer.from_pretrained('gpt2-small')


Loaded pretrained model gpt2-small into HookedTransformer


In [41]:
# layer = 1
# prompt_pos = "Yes, I talk about wedding constantly"
# prompt_neg = "I do not talk about wedding constantly"
# prompt_pos = "Love "
# prompt_neg = "Hate"
prompt_pos = "Anger"
prompt_neg = "Calm"

In [42]:
logits, pos_cache = model.run_with_cache(prompt_pos)
# h_p = pos_cache["resid_pre", layer]

logits, neg_cache = model.run_with_cache(prompt_neg)
# h_n = neg_cache["resid_pre", layer]

# print(h_p.shape, h_n.shape)
# steering = h_p - h_n
# steering.shape

In [43]:
# c = 5
def residual_stream_patching_hook(
    resid,
    hook,
    c
):
    h_p = pos_cache[hook.name]
    h_n = neg_cache[hook.name]
    steering = h_p - h_n

    len_steering = steering.shape[1]
    
    # resid shape is (batch, pos, d_model)
    # print(resid.shape, steering.shape)
    resid[:, :len_steering, :] =  resid[:, :len_steering, :] + c * steering

    return resid

In [44]:
text = "I think you're"
n_samples = 7
hook_name = utils.get_act_name("resid_pre", 7)

with model.hooks(fwd_hooks=[(hook_name, partial(residual_stream_patching_hook, c=5))]):
    # model('testing testing asdf;lkjasdf;l k')
    for i in range(n_samples):
        print("============")
        output = model.generate(text, prepend_bos=True, use_past_kv_cache=False, max_new_tokens=20, verbose=False)
        print(output)

I think you're a fucking monster.

Chairperson: Question marks; for those of you on your right,
I think you're either willfully ignorant or willfully naive, both are completely justified under the circumstances. Even when you think that
I think you're smiling because of my following that ranked this story far below associated research projects. I started tracking it because
I think you're lacking on challenging things herself. Big reveal, is that second half of your comic Mr. Boss?
I think you're correct about a simple key mashing of multiple decimal places: ;. isalphithic if necessary
I think you're going to have to pay attention to your community if you're a chip guy. Addition, subtract
I think you're right that people can't control who they choose to vote for, and I suspect those few who choose


In [45]:
love_words = ["love", "like", "adore", "enjoy", "appreciate", "cherish", "admire", "care", "fancy", "favor", "prefer"]
hate_words = ["hate", "dislike", "detest", "abhor", "despise", "scorn", "loathe", "despise", "fuck you"]

def compute_metric(positive_words, layer, n_samples, factor):
    count = 0
    hook_name = utils.get_act_name("resid_pre", layer)

    with model.hooks(fwd_hooks=[(hook_name, partial(residual_stream_patching_hook, c=factor))]):
        for i in range(n_samples):
            output = model.generate(text, prepend_bos=True, use_past_kv_cache=False, max_new_tokens=25, verbose=False)
            for word in positive_words:
                if word in output.lower():
                    count += 1
                    break
        
    return count/n_samples

In [46]:
for l in range(model.cfg.n_layers):
    n_samples = 10
    score = compute_metric(hate_words, l, n_samples, factor=10)
    print(f"layer: {l}, socre: {score}")

KeyboardInterrupt: 

In [47]:
layer = 7 # pick a layer you want.

# sparse_autoencoder = SparseAutoencoder.from_pretrained(
#     "gpt2-small-res-jb", f"blocks.{layer}.hook_resid_pre"
# )

In [48]:
hook_point = f"blocks.{layer}.hook_resid_pre"

saes, sparsities = get_gpt2_res_jb_saes(hook_point)

print(saes.keys())
sae = saes[hook_point]

100%|██████████| 1/1 [00:00<00:00,  1.17it/s]

dict_keys(['blocks.7.hook_resid_pre'])


In [49]:
### cache angner and then pass through sae, find anger feature, add anger feature during the forward pass.

hook_name = utils.get_act_name("resid_pre", layer)



In [50]:

logits, cache = model.run_with_cache("Anger")
anger_hidden_state = cache[hook_name][0, -1, :].unsqueeze(0)

feature_acts = sae(anger_hidden_state).feature_acts[0]
top_values, top_ids = torch.topk(feature_acts, 10)
print(top_values)
print(top_ids)

tensor([18.4649, 16.4535, 12.0989, 11.0684,  7.7472,  7.2738,  5.0492,  4.7868,
         4.7161,  4.6675])
tensor([16077, 21456,  6857, 23357, 19453, 14237, 12147, 21901, 20881,  9111])


In [51]:
logits, cache = model.run_with_cache("Sad")
anger_hidden_state = cache[hook_name][0, -1, :].unsqueeze(0)

feature_acts = sae(anger_hidden_state).feature_acts[0]
top_values, top_ids = torch.topk(feature_acts, 10)
print(top_values)
print(top_ids)

tensor([39.3675, 16.9168, 15.9074,  5.2266,  5.0246,  3.7320,  3.0697,  3.0380,
         2.8858,  2.3728])
tensor([ 2045, 21248, 20750, 16744, 20385,   667,  4879,  9875, 16910,  8835])


In [52]:
print((feature_acts != 0).sum())

tensor(32)


In [53]:
l1_contribution = top_values[0]/feature_acts.sum()
l1_contribution

tensor(0.3290)

In [54]:
target_feature = top_ids[0]

# only top feature
steering = sae.W_dec[top_ids[0]] * top_values[0]   #sae.W_dec[top_ids[:5]].sum(dim=0) # shape: [d_model]

# top 5 features
# steering = torch.stack([sae.W_dec[top_ids[i]] * top_values[i] for i in range(5)]).sum(dim=0)

steering.shape

torch.Size([768])

In [55]:
# c = 5
def residual_stream_patching_hook_sae(
    resid,
    hook,
    c,
    pos
):    
    # resid shape is (batch, pos, d_model)
    # print(resid.shape, steering.shape)
    if pos < resid.shape[1]:
        resid[:, pos, :] = resid[:, pos, :] + c * steering

    return resid


text = "I think you're"
n_samples = 7

with model.hooks(fwd_hooks=[(hook_name, partial(residual_stream_patching_hook_sae, c=1))]):
    # model('testing testing asdf;lkjasdf;l k')
    for i in range(n_samples):
        print("============")
        output = model.generate(text, prepend_bos=True, use_past_kv_cache=False, max_new_tokens=20, verbose=False)
        print(output)

TypeError: residual_stream_patching_hook_sae() missing 1 required positional argument: 'pos'

In [57]:
love_words = ["love", "like", "adore", "enjoy", "appreciate", "cherish", "admire", "care", "fancy", "favor", "prefer"]
angry_words = ["angry", "mad", "pissed", "irritated", "annoyed", "frustrated", "enraged", "furious", "infuriated", "outraged", "resentful", "agitated", "aggravated", "bitter", "hate", "fuck"]

def compute_metric(related_words, n_samples, coef, pos, max_new_tokens=20):
    count = 0

    with model.hooks(fwd_hooks=[(hook_name, partial(residual_stream_patching_hook_sae, c=coef, pos=pos))]):
        for i in range(n_samples):
            output = model.generate(text, prepend_bos=True, use_past_kv_cache=False, max_new_tokens=max_new_tokens, verbose=False)
            for word in related_words:
                if word in output.lower():
                    count += 1
                    break
        
    return count/n_samples


n_samples = 20
max_new_tokens = 20
n_positions = 10
cs = [0, 0.5, 1, 5, 10, 20]
score_matrix = torch.zeros((n_positions, len(cs)))
for pos in range(n_positions):
    for ci, c in enumerate(cs):
        score = compute_metric(angry_words, n_samples, c, pos, max_new_tokens)
        score_matrix[pos, ci] = score

        print(f'pos: {pos}, c: {c}, socre: {score}')

pos: 0, c: 0, socre: 0.05
pos: 0, c: 0.5, socre: 0.05
pos: 0, c: 1, socre: 0.05
pos: 0, c: 5, socre: 0.0


IndexError: index 4 is out of bounds for dimension 1 with size 4

In [ ]:
fig = px.imshow(score_matrix)
fig.show()


In [ ]:

sum_over_pos = score_matrix.sum(0)
sum_over_pos.shape

px.line(y=sum_over_pos, x=[0.5, 1, 5, 10], title="Sum of scores over positions", markers=True).show()

In [ ]:
sum_over_pos = score_matrix.sum(0)
sum_over_pos.shape

px.line(y=score_matrix.sum(1), x=list(range(10)), title="Sum of scores over positions", markers=True).show()

In [ ]:

model.to_str_tokens(text)

['<|endoftext|>', 'I', ' think', ' you', "'re"]